<div style="text-align: center;">
    <h1> <font style="bold"> Analisis Predictivo Avanzado </font></h1>
    <h2><font style="bold">Trabajo práctico </font></h2>
    <h3><font style="bold">Integrantes:</font></h3>
</div>
<div style="display: flex; justify-content: center;">
    <h4><ul>
        <li>Noguera Abril</li>
        <li>Arbues Lucas</li>
        <li>Alfie Agustin</li>
        </ul>
    </h4>
</div>

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from scipy import stats

# Funciones: 
from funciones import encoding, balance, Optimization

In [27]:
df = pd.read_pickle('df.pkl')

## Encoding:

In [28]:
## HACER PIPELINE PROBANDO LOS DISTINTOS ENCODINGS. 
dfOHE =  pd.concat([encoding(df[['category_id', 'domain_id', 'item_id','name','conversion']], ['category_id', 'domain_id', 'item_id','name'], 'TE'),
                    encoding(df.drop(['category_id', 'domain_id', 'item_id','name','conversion'], axis=1), ['listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp'], 'OHE')], axis=1)
dfLE = encoding(df, ['category_id', 'domain_id', 'item_id','name', 'listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp'], 'LE')
dfOE = encoding(df, ['category_id', 'domain_id', 'item_id','name', 'listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp'], 'OE')
dfFE = encoding(df, ['category_id', 'domain_id', 'item_id','name', 'listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp'], 'FE')
dfTE = encoding(df, ['category_id', 'domain_id', 'item_id','name', 'listing_type_id', 'logistic_type', 'platform', 'cat_warranty', 'is_pdp'], 'TE')

# Exploración e Implementación del Modelo 

Una vez finalizado el analisis exploratorio de datos (EDA), se procede a la exploracion e implementacion del modelo.

- Train: 80%
- Val: 20%
- Test: de testeo.

In [29]:
df = dfOHE

In [30]:
# Split into training and evaluation samples
train_data = df[df["ROW_ID"].isna()]
test_data = df[df["ROW_ID"].notna()]

In [31]:
train_data = train_data.drop(columns=["ROW_ID"])
X_train, X_val, y_train, y_val = train_test_split(train_data.drop(columns=["conversion"]), train_data['conversion'], test_size=0.2)

In [32]:
X_test = test_data.drop(columns=["conversion", "ROW_ID"])

## Balanceo:

In [33]:
print((y_train == 1).value_counts())

conversion
False    131246
True      13362
Name: count, dtype: int64


In [34]:
# OS, US, LT, NM y SM
X_train, y_train = balance(X_train, y_train, 'SM')

## Optimización

In [35]:
model = XGBClassifier(n_jobs=-1, use_label_encoder=False)

/Users/anoguera/anaconda3/envs/itba_apa_env/lib/python3.8/site-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [36]:
grid = {'max_depth': stats.randint(1,20),
              'learning_rate': stats.uniform(0.01, 0.5),
              'subsample': stats.uniform(0.1, 1),
              'colsample_bytree': stats.uniform(0.1, 1),
              'min_child_weight': stats.randint(1,4),
              'n_estimators': stats.randint(50, 1000)
             }

In [37]:
search_spaces = {
    'n_estimators': (10, 200),
    'max_depth': (1, 32),
    'min_samples_split': (2, 20),
}

### Random Search:

In [38]:
model = XGBClassifier(n_jobs=-1, use_label_encoder=False)

In [39]:
grid = {'max_depth': stats.randint(1,20),
              'learning_rate': stats.uniform(0.01, 0.5),
              'subsample': stats.uniform(0.1, 1),
              'colsample_bytree': stats.uniform(0.1, 1),
              'min_child_weight': stats.randint(1,4),
              'n_estimators': stats.randint(50, 1000)
             }

In [40]:
search_spaces = {
    'n_estimators': (10, 200),
    'max_depth': (1, 32),
    'min_samples_split': (2, 20),
}

In [41]:
optimizer = Optimization(model=model, param_grid=grid, n_iter=10, cv=5, random_state=42)

In [42]:
%%capture
RS = optimizer.random_search(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
best_params_random = RS.best_params_
print("Best hyperparameters (Random Search):", best_params_random)

Best hyperparameters (Random Search): {'colsample_bytree': 0.6247564316322378, 'learning_rate': 0.2259725093210579, 'max_depth': 17, 'min_child_weight': 3, 'n_estimators': 108, 'subsample': 0.49986097171525545}


In [ ]:
RS.best_score_

0.9339047970548601

In [ ]:
# Predict on the evaluation set
y_pred = RS.predict_proba(X_test)[:,1]
y_pred

array([8.84840556e-05, 3.23965505e-05, 2.33385782e-03, ...,
       1.40941875e-05, 1.21960649e-04, 9.43253963e-06], dtype=float32)

In [ ]:
# Make the submission file
submission_df = pd.DataFrame({"ROW_ID": test_data["ROW_ID"], "conversion": y_pred})
submission_df["ROW_ID"] = submission_df["ROW_ID"].astype(int)
submission_df.to_csv("xg_boostRS.csv", sep=",", index=False)